In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import gridspec
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Layer, concatenate
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.layers import BatchNormalization


plt.rc('font', size=20)
plt.rcParams["font.family"] = "serif"

2024-06-11 14:55:03.213710: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 14:55:04.616013: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [26]:
normalize = True
N = 10**5
n_moments = 2
mylambda = []
n_bootstraps = 10

In [27]:
moment = 2
obs = 'm'

In [28]:
#load and normalize the data
data = np.load('../InfiniteUnfolding/rawdata.npz')
substructure_variables = ['pT', 'w', 'q', 'm', 'r', 'tau1s', 'tau2s']
data_streams = ['_true', '_true_alt', '_reco', '_reco_alt']
n_variables = len(substructure_variables)


normalize = False
    
for var_name in data.files:
    globals()[var_name] = data[var_name]
    
if normalize:
    for var_name in substructure_variables:
        mu = np.mean(globals()[var_name+data_streams[0]])
        sig = np.std(globals()[var_name + data_streams[0]])
        for stream in data_streams:
            globals()[var_name+stream] = (globals()[var_name+stream] - mu)/sig

## Gaussian

In [29]:
rng = np.random.default_rng()

In [48]:
# x_true = rng.normal(0,1, N)
# x_reco = rng.normal(x_true, 0.5)
# x_true_alt = rng.normal(-0.5,1,N)
# x_reco_alt = rng.normal(x_true_alt, 0.5)

x_true = q_true
x_reco = q_reco
x_true_alt = q_true_alt
x_reco_alt = q_reco_alt

In [49]:
xvals = np.concatenate([x_true_alt,x_true])
xvals_reco = np.concatenate([x_reco_alt,x_reco])
yvals = np.concatenate([np.ones(len(x_true_alt)),np.zeros(len(x_true))])

In [50]:
initializer = tf.keras.initializers.RandomUniform(minval=-.5, maxval=.5)

class MyLayer(Layer):

    def __init__(self, myc, **kwargs):
        self.myinit = myc
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self._l = self.add_weight(name='l', 
                                    shape=(n_moments,),
                                    initializer=tf.keras.initializers.Constant(self.myinit), 
                                    trainable=True)
        
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return tf.exp(sum([self._l[i]* x**(i+1) for i in range(n_moments)]))

In [51]:
def weighted_binary_crossentropy(y_true, y_pred):
    weights = tf.gather(y_true, [1], axis=1) # event weights
    y_true = tf.gather(y_true, [0], axis=1) # actual y_true for loss

    weights_1 = K.sum(y_true*weights)
    weights_0 = K.sum((1-y_true)*weights)

    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    t_loss = -weights * ((y_true) * K.log(y_pred)/weights_1 +
                         (1 - y_true) * K.log(1 - y_pred)/weights_0)
    return K.mean(t_loss)

def weighted_binary_crossentropy_GAN(y_true, y_pred):
    weights = tf.gather(y_pred, [1], axis=1) # event weights
    y_pred = tf.gather(y_pred, [0], axis=1) # actual y_pred for loss

    weights_1 = K.sum(y_true*weights)
    weights_0 = K.sum((1-y_true)*weights)

    #tf.print("weights",weights_0,weights_1)

    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    t_loss = weights * ((1 - y_true) * K.log(1 - y_pred)/weights_0)
    return K.mean(t_loss)

In [52]:
for btsrp in range(n_bootstraps):
    continue

In [53]:
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import concatenate

initializer = tf.keras.initializers.RandomUniform(minval=-5., maxval=5.)

class MyLayer(Layer):

    def __init__(self, myc, **kwargs):
        self.myinit = myc
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self._lambda0 = self.add_weight(name='lambda0', 
                                    shape=(1,),
                                    initializer=tf.keras.initializers.Constant(self.myinit), 
                                    trainable=True)
        self._lambda1 = self.add_weight(name='lambda1', 
                                    shape=(1,),
                                    initializer=tf.keras.initializers.Constant(self.myinit), 
                                    trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        #return tf.exp(self._lambda1 * x + self._lambda0)
        return tf.exp(self._lambda0 * x + self._lambda1 * x**2)


def weighted_binary_crossentropy(y_true, y_pred):
    weights = tf.gather(y_true, [1], axis=1) # event weights
    y_true = tf.gather(y_true, [0], axis=1) # actual y_true for loss
    
    weights_1 = K.sum(y_true*weights)
    weights_0 = K.sum((1-y_true)*weights)
    
    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    t_loss = -weights * ((y_true) * K.log(y_pred)/weights_1 +
                         (1 - y_true) * K.log(1 - y_pred)/weights_0)
    return K.mean(t_loss)


def weighted_binary_crossentropy_GAN(y_true, y_pred):
    weights = tf.gather(y_pred, [1], axis=1) # event weights
    y_pred = tf.gather(y_pred, [0], axis=1) # actual y_pred for loss
    
    weights_1 = K.sum(y_true*weights)
    weights_0 = K.sum((1-y_true)*weights)
    
    #tf.print("weights",weights_0,weights_1)
    
    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    t_loss = weights * ((1 - y_true) * K.log(1 - y_pred)/weights_0)
    return K.mean(t_loss)

In [61]:
mylambda = []
for btstrp in range(1
                    #n_bootstraps
                   ):
    print(f"{btstrp = }")
    myc = rng.normal(0, 1)
    mymodel_inputtest = Input(shape=(1,))
    mymodel_test = MyLayer(myc)(mymodel_inputtest)
    model_generator = Model(mymodel_inputtest, mymodel_test)
    
    inputs_disc = Input((1, ))
    hidden_layer_1_disc = Dense(50, activation='relu')(inputs_disc)
    hidden_layer_2_disc = Dense(50, activation='relu')(hidden_layer_1_disc)
    hidden_layer_3_disc = Dense(50, activation='relu')(hidden_layer_2_disc)
    outputs_disc = Dense(1, activation='sigmoid')(hidden_layer_3_disc)
    model_discrimantor = Model(inputs=inputs_disc, outputs=outputs_disc)
    model_discrimantor.compile(loss=weighted_binary_crossentropy, optimizer='adam')

    model_discrimantor.trainable = False
    mymodel_gan = Input(shape=(1,))
    gan_model = Model(inputs=mymodel_gan,outputs=concatenate([model_discrimantor(mymodel_gan),model_generator(mymodel_gan)]))

    gan_model.compile(loss=weighted_binary_crossentropy_GAN, optimizer='adam')
    
    xvals_1 = np.concatenate([x_true_alt,x_true])
    yvals_1 = np.concatenate([np.ones(len(x_true_alt)),np.zeros(len(x_true))])
    
    X_train_1, X_test_1, Y_train_1, Y_test_1 = train_test_split(xvals_1, yvals_1)
    
    n_epochs = 50
    n_batch = 128*100
    n_batches = len(X_train_1) // n_batch
    
    for i in range(n_epochs):
        lambdasum = np.log(model_generator.predict(np.array([1.])))
        lambdasum2 = np.log(model_generator.predict(np.array([2.])))
        mylambda1 = (lambdasum2-2*lambdasum)/2
        mylambda0 = lambdasum - mylambda1
        print("on epoch=",i,mylambda0,mylambda1)
        #print("  ",np.sum(model_generator.predict(X_train_1,batch_size=1000)))
        for j in range(n_batches):
            X_batch = X_train_1[j*n_batch:(j+1)*n_batch]
            Y_batch = Y_train_1[j*n_batch:(j+1)*n_batch]
            W_batch = model_generator(X_batch)
            W_batch = np.array(W_batch).flatten()
            W_batch[Y_batch==1] = 1
            #W_batch[Y_batch==0] = 1
            
            Y_batch_2 = np.stack((Y_batch, W_batch), axis=1)
            
            model_discrimantor.train_on_batch(X_batch, Y_batch_2)
            
            #print("      ",j,np.sum(model_generator.predict(X_batch,batch_size=1000)),np.log(model_generator.predict([1.]))-np.log(model_generator.predict([0.])),np.log(model_generator.predict([0.])))
            
            gan_model.train_on_batch(X_batch[Y_batch==0],np.zeros(len(X_batch[Y_batch==0])))
    mylambda += [mylambda0[0, 0], mylambda1[0, 0]]

btstrp = 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
on epoch= 0 [[0.18000662]] [[0.18000638]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
on epoch= 1 [[0.18000817]] [[0.18000636]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
on epoch= 2 [[0.18001086]] [[0.18000472]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
on epoch= 3 [[0.1800142]] [[0.18000305]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
on epoch= 4 [[0.18001789]] [[0.1800012]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
on epoch= 5 [[0.1800223]] [[0.17999944]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
on epoch= 6 [[0.18002737]] [[0.17999777]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
on epoch= 7 [[0.18003291]] [[0.17999586]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━

In [62]:
lambdas = np.reshape(mylambda, (len(mylambda)//2, 2))

In [63]:
np.std(lambdas, axis = 0)

array([0., 0.], dtype=float32)

## Truth and Gen

In [92]:
n_bootstraps = 100
for var in substructure_variables:
    for stream in ['_true', '_true_alt']:
        globals()[f'bootstrapped_{var}{stream}'] = rng.choice(globals()[var + stream], 
                                                              replace = True,
                                                              size = (N, n_bootstraps))

In [93]:
np.mean(bootstrapped_w_true, axis = 0).shape

(100,)

In [102]:
def bold(type): sys.stdout.write("\033[1m" + type + "\033[0m \n")

In [105]:
for var in substructure_variables:
    for stream in ['_true', '_true_alt']:
        bold(var + stream)
        for moment in [1, 2]:
            print(f"E(var^{moment}) error = ", np.std(np.mean(globals()[f'bootstrapped_{var}{stream}']**moment, axis = 0)))

pT_true 
E(var^1) error =  0.2950905528869407
E(var^2) error =  220.3026561474027
pT_true_alt 
E(var^1) error =  0.29130622114907967
E(var^2) error =  204.54475261585955
w_true 
E(var^1) error =  0.0002577255459616071
E(var^2) error =  0.0001134367122041186
w_true_alt 
E(var^1) error =  0.00029918196024106996
E(var^2) error =  0.00012834982830220426
q_true 
E(var^1) error =  0.00037477005907097204
E(var^2) error =  8.213229241278366e-05
q_true_alt 
E(var^1) error =  0.0003801314767742829
E(var^2) error =  7.362636137695333e-05
m_true 
E(var^1) error =  0.03035685487560021
E(var^2) error =  2.220650073243317
m_true_alt 
E(var^1) error =  0.043236974888872175
E(var^2) error =  3.049982190476556
r_true 
E(var^1) error =  0.0006965746676179559
E(var^2) error =  0.0008266905751036265
r_true_alt 
E(var^1) error =  0.0006344223936822807
E(var^2) error =  0.0007389082948154491
tau1s_true 
E(var^1) error =  0.00027520087559086947
E(var^2) error =  0.00011570232458948676
tau1s_true_alt 
E(var^1)

# W

In [91]:
lambdaa = 2.2631578947368416
lambdab = 4.157894736842105
widtha =  0.0430505
widthb =  0.0431201
x_true = w_true
x_true_alt = w_true_alt

In [98]:
lambda0down = lambdaa - widtha
lambda1down = lambdab - widthb
weights_down = np.exp(lambda1down*x_true**2+lambda0down*x_true)*len(x_true_alt)/np.sum(np.exp(lambda1down*x_true**2+lambda0down*x_true))
down = np.average(x_true_alt, weights = weights_down)
lambda0up = lambdaa + widtha
lambda1up = lambdab + widthb
weights_up = np.exp(lambda1up*x_true**2+lambda0up*x_true)*len(x_true_alt)/np.sum(np.exp(lambda1up*x_true**2+lambda0up*x_true))
up = np.average(x_true_alt, weights = weights_up)
print(np.abs(up - down))

5.137831846246943e-06


In [101]:
np.sqrt(0.0029**2 + 5.137831846246943e-06**2)

0.002900004551257822

In [102]:
lambda0down = lambdaa - widtha
lambda1down = lambdab - widthb
weights_down = np.exp(lambda1down*x_true**2+lambda0down*x_true)*len(x_true_alt)/np.sum(np.exp(lambda1down*x_true**2+lambda0down*x_true))
down = np.average(x_true_alt**2, weights = weights_down)
lambda0up = lambdaa + widtha
lambda1up = lambdab + widthb
weights_up = np.exp(lambda1up*x_true**2+lambda0up*x_true)*len(x_true_alt)/np.sum(np.exp(lambda1up*x_true**2+lambda0up*x_true))
up = np.average(x_true_alt**2, weights = weights_up)
print(np.abs(up - down))

2.4362108506717495e-06


In [104]:
np.sqrt(0.00128**2 + 2.4362108506717495e-06**2)

0.001280002318405443

# Q

In [107]:
lambdaa = -0.6122449
lambdab = -4.69387755
widtha =  0.18031329
widthb =  0.17988884
x_true = q_true
x_true_alt = q_true_alt

In [108]:
lambda0down = lambdaa - widtha
lambda1down = lambdab - widthb
weights_down = np.exp(lambda1down*x_true**2+lambda0down*x_true)*len(x_true_alt)/np.sum(np.exp(lambda1down*x_true**2+lambda0down*x_true))
down = np.average(x_true_alt, weights = weights_down)
lambda0up = lambdaa + widtha
lambda1up = lambdab + widthb
weights_up = np.exp(lambda1up*x_true**2+lambda0up*x_true)*len(x_true_alt)/np.sum(np.exp(lambda1up*x_true**2+lambda0up*x_true))
up = np.average(x_true_alt, weights = weights_up)
print(np.abs(up - down))

5.173522915288137e-06


In [110]:
np.sqrt(0.000038**2 + 5.173522915288137e-06**2)

3.83505585272889e-05

In [111]:
lambda0down = lambdaa - widtha
lambda1down = lambdab - widthb
weights_down = np.exp(lambda1down*x_true**2+lambda0down*x_true)*len(x_true_alt)/np.sum(np.exp(lambda1down*x_true**2+lambda0down*x_true))
down = np.average(x_true_alt**2, weights = weights_down)
lambda0up = lambdaa + widtha
lambda1up = lambdab + widthb
weights_up = np.exp(lambda1up*x_true**2+lambda0up*x_true)*len(x_true_alt)/np.sum(np.exp(lambda1up*x_true**2+lambda0up*x_true))
up = np.average(x_true_alt**2, weights = weights_up)
print(np.abs(up - down))

1.2719198881974703e-07


# M

In [120]:
lambdaa = 0.09230769
lambdab = 0.006
widtha =   0.005
widthb =  0.001
x_true = m_true
x_true_alt = m_true_alt

In [121]:
lambda0down = lambdaa - widtha
lambda1down = lambdab - widthb
weights_down = np.exp(lambda1down*x_true**2+lambda0down*x_true)*len(x_true_alt)/np.sum(np.exp(lambda1down*x_true**2+lambda0down*x_true))
down = np.average(x_true_alt, weights = weights_down)
lambda0up = lambdaa + widtha
lambda1up = lambdab + widthb
weights_up = np.exp(lambda1up*x_true**2+lambda0up*x_true)*len(x_true_alt)/np.sum(np.exp(lambda1up*x_true**2+lambda0up*x_true))
up = np.average(x_true_alt, weights = weights_up)
print(np.abs(up - down))

0.017848425302261717


In [123]:
np.sqrt(0.17848425302261717**2 + 0.43**2)

0.46557129269000425

In [122]:
lambda0down = lambdaa - widtha
lambda1down = lambdab - widthb
weights_down = np.exp(lambda1down*x_true**2+lambda0down*x_true)*len(x_true_alt)/np.sum(np.exp(lambda1down*x_true**2+lambda0down*x_true))
down = np.average(x_true_alt**2, weights = weights_down)
lambda0up = lambdaa + widtha
lambda1up = lambdab + widthb
weights_up = np.exp(lambda1up*x_true**2+lambda0up*x_true)*len(x_true_alt)/np.sum(np.exp(lambda1up*x_true**2+lambda0up*x_true))
up = np.average(x_true_alt**2, weights = weights_up)
print(np.abs(up - down))

1.016655174143807


In [126]:
np.sqrt(35**2 + 10.16655174143807**2)

36.44665655874812